In [1]:
%matplotlib qt5
import sys
sys.path.append('./pdeSolver')

from domains import *
from equations import *
from torch_solvers import *
import torch

import ipywidgets as widgets
from ipywidgets import interact
from mpl_toolkits import mplot3d
import matplotlib.patches as patches
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt

In [2]:
print("Probando interp")
dom_config={"total_time":1.0,"pInf":0.4,"pSup":0.6}
dom=EmptyRoom(dom_config)
eqn_config={"N":1,"nu":0.05,"lam":4.0}
eqn=HJB_LQR_Equation_2D(dom,eqn_config)

solver_params={"initial_lr":0.001,
            "initial_loss_weigths":[2.0,1.0,1.0,1.0],
            "net_size":{"width":30,"depth":3,"weigth_norm":False},
            "logging_interval":10,
            "dtype":torch.float32,
            "dt":0.01,
            "N_max":25,
            "N_samples_per_batch_interior":200,
            "N_samples_per_batch_boundary":40,
            "sample_every":1,
            "alpha":0.09,
            "adaptive_weigths":False}
sol=Interp_PINN_BSDE_solver(eqn,solver_params)



Probando interp


In [7]:
sol.plot_N_agent_sample_path()

In [3]:
sol.train(600)

Comencemos a calcular
1
Li: 0.09841418266296387
Lb: 0.0006737709045410156
Tiempto toal 0.26204895973205566
2
Li: 0.09267091751098633
Lb: 0.0005962848663330078
Tiempto toal 0.2795860767364502
3
Li: 0.0906825065612793
Lb: 0.00054931640625
Tiempto toal 0.277707576751709
4
Li: 0.09751653671264648
Lb: 0.0005617141723632812
Tiempto toal 0.33020782470703125
5
Li: 0.1075289249420166
Lb: 0.0015707015991210938
Tiempto toal 0.31074023246765137
6
Li: 0.10314512252807617
Lb: 0.0006349086761474609
Tiempto toal 0.2967071533203125
7
Li: 0.09302091598510742
Lb: 0.00054168701171875
Tiempto toal 0.28818511962890625
8
Li: 0.09466767311096191
Lb: 0.0005559921264648438
Tiempto toal 0.2933237552642822
9
Li: 0.0931704044342041
Lb: 0.0005035400390625
Tiempto toal 0.2884104251861572
10
Li: 0.10255646705627441
Lb: 0.0005385875701904297
Tiempto toal 0.2972695827484131
Li: 0.09607243537902832
Lb: 0.003363370895385742
Epoch  10  time  3.025930404663086  loss  [9.722609]  L1: 4.821680068969727  L2: 0.030339641496539

/home/carlos/Documentos/Trabajo de grado/Tesis/code/./pdeSolver/torch_solvers.py:457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(training_history)


array([[10, array([9.722609], dtype=float32), 3.025930404663086],
       [20, array([9.68509], dtype=float32), 6.125405311584473],
       [30, array([9.272011], dtype=float32), 9.334819078445435],
       [40, array([9.5821705], dtype=float32), 12.530322074890137],
       [50, array([9.202223], dtype=float32), 15.928580045700073],
       [60, array([9.462687], dtype=float32), 19.221311569213867],
       [70, array([9.3215885], dtype=float32), 22.603657007217407],
       [80, array([8.781427], dtype=float32), 26.0795636177063],
       [90, array([9.0949545], dtype=float32), 29.615532159805298],
       [100, array([8.708914], dtype=float32), 33.090713024139404],
       [110, array([8.432859], dtype=float32), 36.37538266181946],
       [120, array([8.900941], dtype=float32), 39.79157519340515],
       [130, array([8.670815], dtype=float32), 43.23753809928894],
       [140, array([8.535451], dtype=float32), 46.56500864028931],
       [150, array([8.37327], dtype=float32), 49.92069911956787]

In [2]:
sol=load_sol("/home/carlos/Documentos/Trabajo de grado/Tesis/code/LQR_INTERP_N_=1_solution.sol")
sol.load_model("/home/carlos/Documentos/Trabajo de grado/Tesis/code/models/LQR_INTERP_N=1_model.pth")
#dom.plot_N_brownian_paths(np.sqrt(0.05),0.01,2000,0.0,[0.1,0.1],1,False,False)

In [3]:
sol.plot_solution()